In [155]:
import _csv
import math
import re
import gzip
import json
import numpy as np
import cohere
import nltk
nltk.download('punkt')

from cohere.classify import Example
co = cohere.Client('Odj4ClpHI3fobuB9bErJkfzlszJwQ018Z99gUAyE')
KEY = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Adibvafa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [156]:
user_prompt1 = """
My son claims that hes been having extremely violent thoughts and dreams. Not violent like he's hurting someone, but violent thoughts like horrible things happening to his loved ones. He explained one of his dreams the other day and it was so violent it was sickening. It was far beyond anything in a horror movie, he says he can't help these thoughts they just pop up. Please help!
""".strip()

user_prompt2 = """
I went to the gym last night and saw many people with good fitness. However I am fat and obese. So I couldn't stand being there and came home while cry
""".strip()

user_prompt3 = """
I did not get any valentine chocolate today which makes me very sad. Man I want a girlfriend quickly.
""".strip()

user_prompt4 = """
I came home and suddenly, boom!, happy birth day. My parents really surprised day. What a lovely day it was!
""".strip()

user_prompt5 = """
It's important to note that IMDb ratings are not an absolute measure of a movie's quality, but rather a reflection of the collective opinion of IMDb users who have rated the movie.
""".strip()

user_prompt6 = """
The man felt a sense of joy and fulfillment as he accomplished his lifelong goal, but at the same time, he felt a tinge of sadness that the journey was over and that he would have to start a new chapter in his life.""".strip()

In [157]:
def classify_chunks(response, data=True, if_print=False):
    results = {KEY[i]: [] for i in range(len(KEY))}
    for i in range(len(response.classifications)):
        prediction = int(response.classifications[i].prediction)
        confidence = response.classifications[i].confidence
        if confidence >= 0.5:
            results[KEY[prediction]].append(confidence)
        if data:
            data = response.classifications[i].input
            if if_print:
                print(f'Input: {data}')
        if if_print:
            print(f'Class: {prediction}, Confidence: {confidence} \n')
    return results

In [158]:
def feedback(results, if_print=False):
    if if_print:
        print(results, '\n')
    evaluation = []
    for i in range(len(KEY)):
        data = np.array(results[KEY[i]])
        if len(data) == 0:
            data = [0]
        average = np.mean(data)
        n = len(data)
        coefficient = n / (2 + n)
        evaluation.append(coefficient * average * max(data))
    if if_print:
        print(evaluation)
    return KEY[np.argmax(evaluation)]

In [159]:
def split_input(user_input):
    sentences = nltk.sent_tokenize(user_input)
    return sentences

def split_input_self(user_input):
    Replace = [' and ', ' that ', ' thus ', ' then ', ' however ', ' but ']
    for i in range(len(Replace)):
        user_input = user_input.replace(Replace[i], '.')
    cut_user_prompt = re.split(r'[,.!?-]', user_prompt)
    cut_user_prompt = list(filter(None, cut_user_prompt))
    return cut_user_prompt

In [160]:
def main(user_prompt):
    user_chunks = split_input_self(user_prompt)

    response = co.classify(
      model='0c667180-ef42-4c4d-aee8-e6fa29177e43-ft',
      inputs=user_chunks)

    results = classify_chunks(response)
    emotion = feedback(results)
    return emotion

In [161]:
main(user_prompt1)

'anger'